### QA over SQL Data

We will create a Q&A app over tabular data in databases
These app will allow us to ask a question about the data in a database in natural language and get an answer also in natural language


In [1]:
import warnings
from langchain._api import LangChainDeprecationWarning
warnings.simplefilter(
    "ignore",
    category=LangChainDeprecationWarning
)

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
groq_key = os.getenv("GROQ")
groq_key

'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'

In [3]:
# deepseek-r1-distill-llama-70b
from langchain_groq import ChatGroq
groq_model = ChatGroq(
    model="gemma2-9b-it",
    groq_api_key=groq_key)

### Connect wit the database

In [4]:
from langchain_community.utilities import SQLDatabase

sqlite_db_path = "data/street_tree_db.sqlite"
db = SQLDatabase.from_uri(
    f"sqlite:///{sqlite_db_path}"
)

### Convert the question into sql query
### Execute the query
### Use the result to answer the original question

In [5]:
from langchain.chains import create_sql_query_chain
chain = create_sql_query_chain(
        groq_model, db
)
response = chain.invoke(
    {
        "question": "How many species of trees are in San Francisco?"
    }
)
print(response)

Question: How many species of trees are in San Francisco?
SQLQuery: SELECT COUNT(DISTINCT "qSpecies") FROM street_trees


In [ ]:
#SELECT COUNT(DISTINCT "qSpecies") FROM street_trees;

In [7]:
import re
# Regex pattern to extract any SQL query
match = re.search(r"SQLQuery:\s*(.+)$", response, re.DOTALL)
if match:
    sql_query = match.group(1)
    print(sql_query)
else:
    print("SQL query not found.")

SELECT COUNT(DISTINCT "qSpecies") FROM street_trees


In [8]:
db.run(sql_query)

'[(148,)]'

Executing the SQL query

In [13]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
import re
write_query = create_sql_query_chain(
    groq_model, db
)
execute_query = QuerySQLDataBaseTool(
    db=db
)
# Invoke the query generation
response = write_query.invoke({"question": "List 3 species of trees that are present in San Francisco"})

# Step 2: Extract SQL query
match = re.search(r"SQLQuery:\s*(.+)$", response, re.DOTALL)

if match:
    sql_query = match.group(1).strip()
    print("Extracted SQL Query:", sql_query)

    # Step 3: Execute the extracted query
    result = execute_query.invoke({"query": sql_query})
    print("Query Result:", result)
else:
    print("SQL query not found.")

Extracted SQL Query: SELECT DISTINCT "qSpecies" FROM street_trees LIMIT 3
Query Result: [("Arbutus 'Marina' :: Hybrid Strawberry Tree",), ('Afrocarpus gracilior :: Fern Pine',), ("Thuja occidentalis 'Emerald' :: Emerald Arborvitae",)]


Translate SQL response to Natural Language Response

In [14]:
from operator import itemgetter
import re
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, 
    corresponding SQL query, and SQL result, 
    answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)


# Define a function to extract SQL query using regex in LCEL
def extract_sql_query(response):
    match = re.search(r"SQLQuery:\s*(.+)$", response, re.DOTALL)
    return match.group(1).strip() if match else "SQL query not found."

write_query = create_sql_query_chain(groq_model, db)

execute_query = QuerySQLDataBaseTool(db=db)

# LCEL Chain
chain = (
    RunnablePassthrough.assign(query=write_query)
    .assign(extracted_query=itemgetter("query") | StrOutputParser() | extract_sql_query)
    .assign(result=itemgetter("extracted_query") | execute_query)
    | answer_prompt
    | groq_model
    | StrOutputParser()
)

response = chain.invoke({"question": "List 3 species of trees that are present in San Francisco"})

print(response)

Here are 3 species of trees present in San Francisco:

* Arbutus 'Marina' Hybrid Strawberry Tree
* Afrocarpus gracilior Fern Pine
* Thuja occidentalis 'Emerald' Emerald Arborvitae 



